<a href="https://colab.research.google.com/github/yui828/hungman/blob/master/NLP_word2vec_usingmecab_%E5%BE%A9%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 単語のベクトル化
単語のベクトル化は、以下の順に行います。

1. 前処理
2. word2vecモデルを学習

本ノートブックでは、Day1で利用したWikipedia のデータを利用して、単語のベクトル化を行います。

In [ ]:
!pip install gensim
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

In [ ]:
import MeCab
import os
import numpy as np
import pandas as pd
import unicodedata

## データの読み込み

In [ ]:
# データのダウンロード
!wget https://basic-unstructured-data.s3-ap-northeast-1.amazonaws.com/unstructured-data-day2/data.zip
!unzip data.zip

In [ ]:
train = pd.read_csv("data/train.csv")
train.head()

,text,label
0,世界の映画祭で話題沸騰！インドネシアの壮絶バイオレンス・アクション『ザ・レイド』予告編が解禁...,1
1,冬の女子会は特製「薬膳火鍋」に決定！美味しく食べてポッカポカ本格的な寒さを感じるこの季節。「...,2
2,2012年・夏の浴衣スタイルのカギは帯！夏といえば浴衣。「そろそろ新しい浴衣を買おうかな」と...,2
3,インタビュー：武田修宏さん「結婚するならセリエAクラスの女性がいい」少年時代から天才サッカー...,2
4,「ポケモン」や「ONE PIECE」と同じ賞を獲得したキモかわいいキャラが全国に「クサマダラ...,1


In [ ]:
test = pd.read_csv("data/test.csv")
test.head()

,text,label
0,【Sports Watch】元代表二人によるカメルーン戦の予想は？TBSの報道番組「NEWS...,3
1,若槻千夏に激似!? 女子バレーのニューヒロイン新鍋理沙バレーボールのW杯が開幕し、ロンドン五...,3
2,ソーシャルの今が分かる！ヤフーの「話題なう」ってなに？ヤフーは、Twitterなどのソーシャ...,0
3,もう一つのユーロ2012美女ばかりの観客席には選手の彼女も!?連日連夜熱戦が展開されているサ...,3
4,元日本代表監督の岡田武史氏、杭州緑城の監督に決定日本代表の前監督である岡田武史氏が15日、中...,3


## 正規化
---

In [ ]:
train["text"] = train["text"].str.normalize("NFKC")
test["text"] = test["text"].str.normalize("NFKC")

今回は練習のために"「」"と"『』"の区別がないので、統一化してみましょう。

In [ ]:
unify_dic = {
    '『': '「',
    '』': '」'
}


def unify_str(x):
    dic_for_unify = str.maketrans(unify_dic)
    x = x.translate(dic_for_unify)
    return x


In [ ]:
train["text"] = train["text"].apply(unify_str)
test["text"] = test["text"].apply(unify_str)

## その他前処理
---
### 文書と関係のない記号の削除

```In [1]: df["text"][2][:20]
Out[2]: '理性 理性(りせい、→→→)とは、人間に'
```

のような"→"は文書と関係ないと考えることができるので除去する。<br>

In [ ]:
stop_words = ["→", "←", "?", "」", "「", ":", "!"]  # 適当な文字を設定


def remove_stop_words(x):
    for s in stop_words:
        x = x.replace(s, '')
    return x


train["text"] = train["text"].apply(remove_stop_words)
test["text"] = test["text"].apply(remove_stop_words)

In [ ]:
train["text"].head()

0    世界の映画祭で話題沸騰インドネシアの壮絶バイオレンス・アクションザ・レイド予告編が解禁インド...
1    冬の女子会は特製薬膳火鍋に決定美味しく食べてポッカポカ本格的な寒さを感じるこの季節。何が食べ...
2    2012年・夏の浴衣スタイルのカギは帯夏といえば浴衣。そろそろ新しい浴衣を買おうかなと、思っ...
3    インタビュー武田修宏さん結婚するならセリエAクラスの女性がいい少年時代から天才サッカー少年と...
4    ポケモンやONE PIECEと同じ賞を獲得したキモかわいいキャラが全国にクサマダラオオコビト...
Name: text, dtype: object

## 形態素解析
---

mecab を利用した形態素解析

In [ ]:
def get_surfaces(text):
    result = []
    node = mecab.parseToNode(text)
    while node:
        if not node.feature.startswith("BOS/EOS") and \
          not node.feature.startswith("助詞") and \
          not node.feature.startswith("助動詞") and \
          not node.feature.startswith("記号"):
            result.append(node.surface)
        node = node.next
    return " ".join(result)


In [ ]:
mecab = MeCab.Tagger()
# バグ回避用
mecab.parse("")

train['text_tokenized'] = train['text'].apply(get_surfaces)
test['text_tokenized'] = test['text'].apply(get_surfaces)

In [ ]:
train.head()

,text,label,text_tokenized
0,世界の映画祭で話題沸騰インドネシアの壮絶バイオレンス・アクションザ・レイド予告編が解禁インド...,1,世界 映画 祭 話題 沸騰 インドネシア 壮絶 バイオレンス・アクションザ・レイド 予告編 ...
1,冬の女子会は特製薬膳火鍋に決定美味しく食べてポッカポカ本格的な寒さを感じるこの季節。何が食べ...,2,冬 女子 会 特製 薬 膳 火 鍋 決定 美味しく 食べ ポッカポカ 本格 的 寒 さ 感じ...
2,2012年・夏の浴衣スタイルのカギは帯夏といえば浴衣。そろそろ新しい浴衣を買おうかなと、思っ...,2,2012 年 夏 浴衣 スタイル カギ 帯 夏 いえ 浴衣 そろそろ 新しい 浴衣 買お 思...
3,インタビュー武田修宏さん結婚するならセリエAクラスの女性がいい少年時代から天才サッカー少年と...,2,インタビュー 武田 修 宏 さん 結婚 する セ リエ A クラス 女性 いい 少年 時代 ...
4,ポケモンやONE PIECEと同じ賞を獲得したキモかわいいキャラが全国にクサマダラオオコビト...,1,ポケモン ONE PIECE 同じ 賞 獲得 し キモ かわいい キャラ 全国 クサマダラオ...


## 特定の文字列を各文書から削除する

In [ ]:
def remove_stop_words(sentence):
    stop_words = ["その", "ため"] # 適当な文字を設定
    for s in stop_words:
        sentence = sentence.replace(s, '')
    return sentence

In [ ]:
train["text_tokenized"] = train["text_tokenized"].apply(remove_stop_words)
test["text_tokenized"] = test["text_tokenized"].apply(remove_stop_words)

## ライブラリのインポート

In [ ]:
from gensim.models import word2vec

In [ ]:
# 【ご参考】Phrasesを適用し頻出する単語のペアを1フレーズに変換する
# 例えば「データミックス」をjanomeで分かち書きすると、辞書登録がなければ「データ」と「ミックス」に分解されます。<br>
# しかし文書内に「データミックス」が頻出する場合は、「データ」と「ミックス」を「_」でつないで「データ_ミックス」
# という1単語にします。

from gensim.test.utils import datapath
from gensim.models.word2vec import Text8Corpus
from gensim.models.phrases import Phrases, Phraser

sentences = Text8Corpus(datapath('testcorpus.txt'))
phrases = Phrases(sentences, min_count=1, threshold=1)  # train model
phrases[[u'trees', u'graph', u'minors']]  # apply model to sentence
# 'trees'と'graph'が_で繋がり、1単語になる

# 参考URL
# https://radimrehurek.com/gensim/models/phrases.html

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['trees_graph', 'minors']

In [ ]:
train["text_tokenized"].head()

0    世界 映画 祭 話題 沸騰 インドネシア 壮絶 バイオレンス・アクションザ・レイド 予告編 ...
1    冬 女子 会 特製 薬 膳 火 鍋 決定 美味しく 食べ ポッカポカ 本格 的 寒 さ 感じ...
2    2012 年 夏 浴衣 スタイル カギ 帯 夏 いえ 浴衣 そろそろ 新しい 浴衣 買お 思...
3    インタビュー 武田 修 宏 さん 結婚 する セ リエ A クラス 女性 いい 少年 時代 ...
4    ポケモン ONE PIECE 同じ 賞 獲得 し キモ かわいい キャラ 全国 クサマダラオ...
Name: text_tokenized, dtype: object

### gensimのword2vecに渡せる形に変換

In [ ]:
sentences = [token.split(" ") for token in train.text_tokenized]

*   sentencesには2785個のリストがある
*   sentences = [[0行目の文書の単語],[1行目‥],　....]]

In [ ]:
len(sentences)

2785

In [ ]:
sentences[0]

### モデルの学習
gensimのword2vecを利用し以下2モデルの実装を行います
- CBOW(Continuous Bag-of-Words)
- Skip-gram

#### gensimのword2vecの主な引数

|<center>引数</center>|<center>詳細</center>|
| --- | --- |
|<center>sg</center>|<div style="text-align: left;">1を選べばskip-gram、0ならばCBOW</div>|
|<center>size</center>|<div style="text-align: left;">特徴ベクトルの次元数の設定。<br>sizeの値が大きいほど次元数が大きくなるので、表現力が上がる。<br>しかしsizeの値を大きくすると、計算時間がかかったり、メモリ占有率が上がったりする。</div>|
|<center>min_count</center>|<div style="text-align: left;">一定の頻度以下の単語を除外する際の値を設定する。<br>学習データの文書数とも関連する。一定頻度出現する単語を直接確認し、適切だと思われる値を設定する。</div>|
|<center>window</center>|<div style="text-align: left;">学習に利用する周辺単語の範囲を指定する。<br>値を設定する際は文書の特徴を見て判断するのが良い。<br>例えばニュース記事や説明書によって、予測する単語が関連する範囲が変わると思われるため。</div>|
|<center>negative</center>|<div style="text-align: left;">0よりも大きければネガティブサンプリングが用いられる。<br>0であればネガティブサンプリングが適用されない。</div>|


### まずはCBOWモデルの作成をします

In [ ]:
# CBOWモデルの学習
cbow_model = word2vec.Word2Vec(sentences,
                               sg=0,
                               size=250,
                               min_count=10,
                               window=15,
                               seed=1234)

In [ ]:
# 作成したモデルの保存
cbow_model.save("cbow_w2v.model")
# saveしたモデルを読み込む時は
# model = word2vec.Word2Vec.load("./w2v.model")

In [ ]:
# 映画と似たキーワードを見つけていきます。
# ここで記載しているscoreは、単語同士のコサイン類似度です。
pd.DataFrame(cbow_model.wv.most_similar(
    positive=['映画']), columns=["keyword", "score"])

,keyword,score
0,劇場,0.814640
1,キセキ,0.798916
2,ドラマ,0.797846
3,観る,0.794280
4,名作,0.786865
5,ひみ,0.775069
6,観,0.774080
7,カンタ,0.772750
8,エンター,0.770537
9,ベルセルク,0.762268


In [ ]:
# 結婚と似たキーワードを見つけていきます。
pd.DataFrame(cbow_model.wv.most_similar(
    positive=['結婚']), columns=["keyword", "score"])

,keyword,score
0,彼氏,0.898464
1,初対面,0.840336
2,付き合い,0.809384
3,相談,0.808942
4,ダメ,0.790987
5,意外,0.785790
6,寂しい,0.778444
7,振り返り,0.775328
8,浮気,0.773200
9,質問,0.754491


### Skip-gramモデルも作成してみましょう

In [ ]:
# skip-gramモデルの学習
skipgram_model = word2vec.Word2Vec(sentences,
                                   sg=1,
                                   size=250,
                                   min_count=10,
                                   window=15, seed=1234)

In [ ]:
# 作成したモデルの保存
skipgram_model.save("skipgram_w2v.model")
# saveしたモデルを読み込む時は
# model = word2vec.Word2Vec.load("./skipgram_w2v.model")

In [ ]:
# 映画と似たキーワードを見つけていきます。
# ここで記載しているscoreは、単語同士のコサイン類似度です。
pd.DataFrame(skipgram_model.wv.most_similar(
    positive=['映画']), columns=["keyword", "score"])

,keyword,score
0,ウルトラマンサーガ,0.620860
1,ゾンビ,0.618450
2,マサキ,0.600423
3,マダガスカル,0.597838
4,蘇る,0.597154
5,エンタテイメント,0.589182
6,ビビビ,0.588942
7,キツツキ,0.585571
8,GW,0.585375
9,洋画,0.584881


In [ ]:
# 結婚と似たキーワードを見つけていきます。
pd.DataFrame(skipgram_model.wv.most_similar(
    positive=['結婚']), columns=["keyword", "score"])

,keyword,score
0,独身,0.687369
1,出産,0.678187
2,付き合っ,0.677122
3,交際,0.676919
4,バツ,0.662746
5,婚,0.649710
6,白河,0.645941
7,年上,0.642887
8,桃子,0.640592
9,プロポーズ,0.638906


## 文書のベクトル化
---
単語のベクトル化するモデルは、前述のコードで作成しました。
次は作成したword2vecモデルを活用して、文書のベクトル化を行いましょう。
手順としては以下の通りです。
1. 抽出された各単語を、作成したword2Vecによってベクトル化
2. 変換された各単語ベクトルの平均をとり、その結果得られたベクトルを記事のベクトルとする


そして文書のベクトル化ができたら、テキスト分類を行いましょう。
テキスト情報を上記ステップで定量化することで、テキストをカテゴリー別に分類してみましょう

In [ ]:
"""
1. 抽出された各単語を、作成したword2Vecによってベクトル化
2. 変換された各単語ベクトルの平均をとり、その結果得られたベクトルを記事のベクトルとする
上記2ステップを実行する関数を作成
"""

num_features = 250


def avg_document_vector(data, num_features):
    document_vec = np.zeros((len(data), num_features))
    for i, doc_word_list in enumerate(data):
        feature_vec = np.zeros((num_features,), dtype="float32")
        for word in doc_word_list:
            try:
                feature_vec = np.add(
                    feature_vec, skipgram_model.wv.__getitem__(word))
            except:
                pass

        feature_vec = np.divide(feature_vec, len(doc_word_list))
        document_vec[i] = feature_vec
    return document_vec

df["text"]を変換したデータ(sentences)に対して
1. 抽出された各単語を、作成したword2Vecによってベクトル化
2. 変換された各単語ベクトルの平均をとり、その結果得られたベクトルを記事のベクトルとする

を実施する

In [ ]:
# データイメージ参考
sentences[0][:10]

['世界', '映画', '祭', '話題', '沸騰', 'インドネシア', '壮絶', 'バイオレンス・アクションザ・レイド', '予告編', '解禁']

In [ ]:
X = avg_document_vector(data=sentences, num_features=250)

In [ ]:
# 行が文書なので2785
# 列がモデル作成時の特徴ベクトルの次元(size)なので250 となる
X.shape

(2785, 250)

## テストデータの学習済みword2vecを活用した文書ベクトル化

In [ ]:
# gensimのword2vecに渡せる形に変換
test_sentences = [token.split(" ") for token in test.text_tokenized]

In [ ]:
X_test = avg_document_vector(data=test_sentences, num_features=250)

## 分類器の適用・比較
文書のベクトル化ができたら、後はこれまでやってきた機械学習と同様です。

いくつかの分類器で、性能を比較してみましょう。

In [ ]:
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
# Accuracy, Precision/Recall/F-score/Support, Confusion Matrix を表示
def show_evaluation_metrics(y_true, y_pred):
    print("Accuracy:")
    print(accuracy_score(y_true, y_pred))
    print()

    print("Report:")
    print(classification_report(y_true, y_pred))

    print("Confusion matrix:")
    print(confusion_matrix(y_true, y_pred))

### ロジスティック回帰

In [ ]:
clf_lr = LogisticRegression(n_jobs=-1)
clf_lr.fit(X, train["label"])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
clf_lr.predict(X_test)

array([3, 3, 0, 3, 3, 0, 1, 1, 0, 0, 1, 1, 0, 1, 2, 1, 0, 0, 3, 2, 2, 0,
       0, 0, 3, 1, 2, 0, 1, 0, 0, 3, 0, 1, 2, 3, 2, 3, 0, 1, 3, 1, 0, 3,
       1, 3, 2, 0, 2, 3, 1, 3, 1, 0, 1, 0, 3, 0, 3, 1, 3, 1, 1, 1, 0, 3,
       0, 0, 1, 0, 0, 0, 1, 0, 3, 1, 1, 3, 3, 2, 2, 2, 0, 0, 0, 3, 0, 1,
       3, 3, 0, 0, 3, 1, 1, 3, 2, 1, 0, 3, 2, 3, 1, 1, 3, 3, 0, 2, 3, 2,
       1, 1, 3, 0, 2, 3, 1, 0, 1, 0, 0, 2, 3, 1, 3, 2, 1, 2, 3, 0, 2, 2,
       0, 0, 3, 0, 0, 0, 3, 1, 1, 1, 3, 0, 2, 3, 3, 3, 0, 1, 0, 0, 2, 0,
       0, 0, 2, 3, 1, 2, 0, 0, 1, 0, 3, 3, 3, 2, 0, 1, 3, 3, 1, 3, 1, 0,
       1, 1, 2, 1, 3, 1, 3, 3, 2, 2, 2, 2, 0, 1, 2, 1, 3, 3, 0, 2, 3, 3,
       1, 2, 3, 1, 2, 0, 3, 2, 0, 0, 0, 1, 0, 3, 1, 3, 0, 0, 1, 1, 2, 3,
       0, 2, 2, 2, 1, 1, 3, 0, 3, 1, 1, 2, 1, 3, 3, 0, 3, 3, 1, 1, 3, 2,
       1, 0, 3, 0, 0, 2, 1, 3, 0, 3, 2, 0, 0, 1, 3, 1, 0, 0, 0, 0, 2, 0,
       1, 3, 2, 1, 2, 3, 2, 2, 2, 3, 1, 1, 1, 0, 1, 2, 0, 3, 0, 3, 1, 3,
       0, 0, 3, 0, 3, 3, 3, 2, 0, 0, 0, 3, 0, 3, 2,

In [ ]:
y_test_pred = clf_lr.predict(X_test)
show_evaluation_metrics(test["label"], y_test_pred)

Accuracy:
0.9641319942611191

Report:
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       183
           1       0.93      0.98      0.95       173
           2       0.95      0.91      0.93       155
           3       0.99      0.99      0.99       186

    accuracy                           0.96       697
   macro avg       0.96      0.96      0.96       697
weighted avg       0.96      0.96      0.96       697

Confusion matrix:
[[177   1   4   1]
 [  1 169   3   0]
 [  3  11 141   0]
 [  1   0   0 185]]


### Random Forest

In [ ]:
clf_rf = RandomForestClassifier(n_estimators=50, n_jobs=-1)
clf_rf.fit(X, train["label"])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [ ]:
y_test_pred = clf_rf.predict(X_test)
show_evaluation_metrics(test["label"], y_test_pred)

Accuracy:
0.9583931133428981

Report:
              precision    recall  f1-score   support

           0       0.97      0.95      0.96       183
           1       0.93      0.98      0.96       173
           2       0.93      0.90      0.92       155
           3       0.99      0.99      0.99       186

    accuracy                           0.96       697
   macro avg       0.96      0.96      0.96       697
weighted avg       0.96      0.96      0.96       697

Confusion matrix:
[[174   1   8   0]
 [  1 170   2   0]
 [  3  11 140   1]
 [  2   0   0 184]]


In [ ]:
# submitするためのcsvファイルを作成
y_test_pred = clf_lr.predict_proba(test_tfidf)
submissions = pd.DataFrame({"Id": list(test["Id"]), "polarity_flag": y_test_pred[:, 1]})
submissions.to_csv("submission_ny.csv", index=False, header=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')